In [ ]:
import cv2
import numpy as np
import time
import google.colab.files as files
from google.colab.patches import cv2_imshow
# loaded=files.upload()
video="/content/1860079-uhd_2560_1440_25fps.mp4"
# Load YOLOv3 Tiny model
net = cv2.dnn.readNet("/content/yolov3.weights", "/content/yolov3.cfg")

# Load the COCO names (object classes)
classes = []
with open("/content/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Assign random colors to each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Load video or webcam
cap = cv2.VideoCapture(video)
if not cap.isOpened():
    print("Error: Could not open video stream or file.")
    exit()

# Set font and initialize variables
font = cv2.FONT_HERSHEY_SIMPLEX
starting_time = time.time()
frame_id = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("End of video")
        break
    frame_id += 1

    # Get frame dimensions
    height, width, channels = frame.shape

    # Preprocess the image (YOLO requires this specific format)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Perform forward pass to get output layers
    outs = net.forward(output_layers)

    # Variables to store detection data
    class_ids = []
    confidences = []
    boxes = []

    # Iterate through each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.2:  # Only consider detections above confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate top-left corner
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Save detection data
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression to remove duplicates
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the detected objects
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {round(confidence, 2)}", (x, y - 10), font, 0.5, color, 2)

    cv2_imshow(frame)
    # Break the loop if 'ESC' key is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()
